<a href="https://colab.research.google.com/github/hannapalya/anomaly_detection_syndromic/blob/main/LSTM.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#!/usr/bin/env python3
"""
LSTM Autoencoder — Tail-only metrics, using IsolationForest per-sim splits
- Uses IsolationForest_per_sim_val.csv and IsolationForest_per_sim_test.csv to define splits.
  If a 'split' column exists with values {train,val,test}, those are used directly
  (supports 60/20/20 from IF). If absent, rows from the *_val.csv are 'val' and
  *_test.csv are 'test', and TRAIN = remaining sims.
- Unsupervised training on first 6 years from TRAIN sims
- Validation tuner (mix + threshold), optional guard-rail threshold (train 95th pct for chosen mix)
- Evaluates tail-only comparator metrics (last 49 weeks) with FULL per-day coverage (no front padding)

Files written:
  - LSTM_AE_tail_only_metrics.csv
  - (optional confirmation) lstm_val_indices_sig{S}.csv, lstm_test_indices_sig{S}.csv
"""

import os
import re
import numpy as np
import pandas as pd
import tensorflow as tf
from sklearn.metrics import confusion_matrix

# ========== CONFIG ==========
DATA_DIR      = ""   # <-- set folder with simulated_totals_sig{S}.csv & simulated_outbreaks_sig{S}.csv
VAL_SPLIT_CSV = "IsolationForest_big_medium_per_sim_val.csv"
TEST_SPLIT_CSV= "IsolationForest_big_medium_per_sim_test.csv"

SIGNALS       = list(range(5, 9))
DAYS_PER_YEAR = 364
SEQ, STRIDE   = 14, 1
TRAIN_YEARS   = 6
TRAIN_DAYS    = TRAIN_YEARS * DAYS_PER_YEAR
TAIL_DAYS     = 49 * 7   # 343 (last 49 weeks)

# Training
EPOCHS        = 60
BATCH_SIZE    = 128
LEARNING_RATE = 1e-3
PATIENCE      = 7
MIN_EPOCHS    = 20
RNG_STATE     = 42

# Validation tuning
SPEC_TARGET       = 0.97
TUNE_WEIGHT_SENS  = 2.0
TUNE_WEIGHT_SPEC  = 3.0

MIX_GRID = (
    (1.00, 0.00), (0.95, 0.05), (0.90, 0.10),
    (0.80, 0.20), (0.70, 0.30), (0.60, 0.40),
    (0.40, 0.60), (0.30, 0.70), (0.20, 0.80),
    (0.10, 0.90), (0.05, 0.95), (0.00, 1.00),
)
ANOMALY_MIX_MSE   = 0.8
ANOMALY_MIX_MAX   = 0.2

np.random.seed(RNG_STATE)
tf.random.set_seed(RNG_STATE)

print("GPU Available:", tf.config.list_physical_devices("GPU"))
gpus = tf.config.list_physical_devices('GPU')
if gpus:
    try:
        tf.config.experimental.set_memory_growth(gpus[0], True)
    except Exception as e:
        print("Could not set memory growth:", e)

# ========== SPLIT HELPERS ==========
SIM_RE = re.compile(r"^sig(?P<sig>\d+)_sim(?P<idx>\d+)$")

def parse_sim_name(sim_name):
    m = SIM_RE.match(sim_name)
    if not m:
        return None, None
    return int(m.group("sig")), int(m.group("idx"))

def load_if_splits(val_csv, test_csv):
    """
    Return dict: signal -> {'train': set(sim_names), 'val': set(sim_names), 'test': set(sim_names)}.

    If CSVs include a 'split' column with values in {train,val,test}, those are used directly
    (supports a single large-file style or explicit 60/20/20 lists stored across the two CSVs).
    If 'split' is absent, rows from `val_csv` are treated as 'val', rows from `test_csv` as 'test',
    and TRAIN will be derived as the leftover sims present in the data.
    """
    def _load(path, default_split):
        df = pd.read_csv(path)
        if "signal" not in df.columns or "sim" not in df.columns:
            raise ValueError(f"{path} must contain columns ['signal','sim', ...]")
        split_col = df["split"].astype(str).str.lower() if "split" in df.columns else pd.Series([default_split]*len(df))
        return pd.DataFrame({
            "signal": df["signal"].astype(int),
            "sim": df["sim"].astype(str),
            "split": split_col
        })

    dfv = _load(val_csv,  default_split="val")
    dft = _load(test_csv, default_split="test")
    df  = pd.concat([dfv, dft], ignore_index=True)

    by_signal = {}
    for _, row in df.iterrows():
        sig = int(row["signal"]); sim = str(row["sim"]); split = str(row["split"]).lower()
        d = by_signal.setdefault(sig, {"train": set(), "val": set(), "test": set()})
        if split in {"train", "val", "test"}:
            d[split].add(sim)
    return by_signal

# ========== DATA HELPERS ==========
def load_signal(sig):
    x_path = os.path.join(DATA_DIR, f"simulated_totals_sig{sig}.csv")
    y_path = os.path.join(DATA_DIR, f"simulated_outbreaks_sig{sig}.csv")
    if not (os.path.exists(x_path) and os.path.exists(y_path)):
        raise FileNotFoundError(f"Missing CSVs for signal {sig}: {x_path}, {y_path}")
    X = pd.read_csv(x_path)
    Y = pd.read_csv(y_path)
    for c in list(X.columns):
        if c.lower() in ("date", "ds", "timestamp"):
            X = X.drop(columns=[c])
            if c in Y.columns:
                Y = Y.drop(columns=[c])
            break
    X = X.apply(pd.to_numeric, errors="coerce")
    Y = (Y.apply(pd.to_numeric, errors="coerce") > 0).astype(int)
    return X, Y

def make_seq_labels(series, labels, seq_len=SEQ, stride=STRIDE):
    X, Y = [], []
    for i in range(0, len(series) - seq_len + 1, stride):
        X.append(series[i:i+seq_len])
        Y.append(labels[i+seq_len-1])
    return np.array(X), np.array(Y)

# ========== MODEL ==========
def build_lstm_autoencoder(seq_len=SEQ, learning_rate=LEARNING_RATE):
    inputs = tf.keras.Input(shape=(seq_len, 1))
    x = tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(128, return_sequences=True))(inputs)
    x = tf.keras.layers.BatchNormalization()(x); x = tf.keras.layers.Dropout(0.2)(x)
    x = tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(64, return_sequences=True))(x)
    x = tf.keras.layers.BatchNormalization()(x); x = tf.keras.layers.Dropout(0.2)(x)
    x = tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(32, return_sequences=False))(x)
    x = tf.keras.layers.BatchNormalization()(x); x = tf.keras.layers.Dropout(0.3)(x)
    bottleneck = tf.keras.layers.Dense(4, activation="linear")(x)
    x = tf.keras.layers.RepeatVector(seq_len)(bottleneck)
    x = tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(32, return_sequences=True))(x)
    x = tf.keras.layers.BatchNormalization()(x); x = tf.keras.layers.Dropout(0.2)(x)
    x = tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(64, return_sequences=True))(x)
    x = tf.keras.layers.BatchNormalization()(x); x = tf.keras.layers.Dropout(0.2)(x)
    x = tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(128, return_sequences=True))(x)
    x = tf.keras.layers.BatchNormalization()(x); x = tf.keras.layers.Dropout(0.2)(x)
    outputs = tf.keras.layers.TimeDistributed(tf.keras.layers.Dense(1, activation="linear"))(x)
    model = tf.keras.Model(inputs, outputs)
    model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate), loss="huber", metrics=["mse"])
    return model

# ========== TUNING HELPERS ==========
def sens_spec_for_tuning(y_true, y_pred):
    tn, fp, fn, tp = confusion_matrix(y_true, y_pred, labels=[0,1]).ravel()
    s  = tp/(tp+fn) if (tp+fn)>0 else 0.0
    sp = tn/(tn+fp) if (tn+fp)>0 else 0.0
    return s, sp

def tune_aggressive_threshold(y_val, anomaly_scores, spec_target=SPEC_TARGET, w_sens=TUNE_WEIGHT_SENS, w_spec=TUNE_WEIGHT_SPEC):
    best_t, best_score = None, -1.0
    for p in [85,87,89,90,91,92,93,94,95,96,97,98,98.5,99,99.2,99.5]:
        t = np.percentile(anomaly_scores, p)
        yhat = (anomaly_scores >= t).astype(int)
        s, sp = sens_spec_for_tuning(y_val, yhat)
        if sp >= spec_target:
            score = w_sens*s + w_spec*sp
            if score > best_score:
                best_score, best_t = score, float(t)
    if best_t is None:
        for p in [80,82,84,85,86,87,88,89,90,91,92,93,94]:
            t = np.percentile(anomaly_scores, p)
            yhat = (anomaly_scores >= t).astype(int)
            s, sp = sens_spec_for_tuning(y_val, yhat)
            if sp >= spec_target:
                score = w_sens*s + w_spec*sp
                if score > best_score:
                    best_score, best_t = score, float(t)
    if best_t is None:
        best_t = float(np.percentile(anomaly_scores, 80))
        print("WARNING: No threshold achieved target specificity; using conservative 80th pct.")
    return best_t

def tune_mix_and_threshold(Xval, Yval, recon_val,
                           mixes=MIX_GRID, spec_target=SPEC_TARGET,
                           w_sens=TUNE_WEIGHT_SENS, w_spec=TUNE_WEIGHT_SPEC):
    val_mse = np.mean((Xval - recon_val)**2, axis=(1,2))
    val_max = np.max(np.abs(Xval - recon_val), axis=(1,2))
    best = dict(score=-1.0, mix=None, threshold=None, sens=None, spec=None)
    for m_mse, m_max in mixes:
        scores = m_mse*val_mse + m_max*val_max
        thr = tune_aggressive_threshold(Yval, scores, spec_target, w_sens, w_spec)
        yhat = (scores >= thr).astype(int)
        s, sp = sens_spec_for_tuning(Yval, yhat)
        if sp >= spec_target:
            score = w_sens*s + w_spec*sp
            if score > best["score"]:
                best.update(score=score, mix=(m_mse, m_max), threshold=thr, sens=s, spec=sp)
    if best["mix"] is None:
        best_spec = -1.0
        for m_mse, m_max in mixes:
            scores = m_mse*val_mse + m_max*val_max
            thr = float(np.percentile(scores, 80))
            yhat = (scores >= thr).astype(int)
            s, sp = sens_spec_for_tuning(Yval, yhat)
            if sp > best_spec:
                best_spec = sp
                best.update(score=w_sens*s + w_spec*sp, mix=(m_mse, m_max),
                            threshold=thr, sens=s, spec=sp)
        print("NOTE: no mix hit the specificity target; chose the most specific fallback.")
    return best

# ========== COMPARATOR METRICS (TAIL-ONLY) ==========
def compute_fpr_tail(A_tail, O_tail):
    FP = np.sum((A_tail == 1) & (O_tail == 0))
    N0 = np.sum(O_tail == 0)
    return (FP / N0) if N0 > 0 else np.nan

def compute_specificity_tail(A_tail, O_tail):
    TN = np.sum((A_tail == 0) & (O_tail == 0))
    N0 = np.sum(O_tail == 0)
    return (TN / N0) if N0 > 0 else np.nan

def compute_sensitivity_tail(A_tail, O_tail):
    TP = np.sum((A_tail == 1) & (O_tail > 0))
    P  = np.sum(O_tail > 0)
    return (TP / P) if P > 0 else np.nan

def compute_pod_tail(A_tail, O_tail):
    nsim = A_tail.shape[1]
    hits = [np.any((A_tail[:, j] == 1) & (O_tail[:, j] > 0)) for j in range(nsim)]
    return float(np.mean(hits)) if nsim > 0 else np.nan

def compute_timeliness_tail(A_tail, O_tail):
    nsim = A_tail.shape[1]
    score = 0.0
    miss  = 0
    for j in range(nsim):
        o = O_tail[:, j]
        if np.sum(o > 0) == 0:
            miss += 1
            continue
        r_idx = np.where(o > 0)[0]
        r1, r2 = int(r_idx[0]), int(r_idx[-1])
        a = A_tail[:, j]
        hit_idx = np.where((a == 1) & (o > 0))[0]
        if len(hit_idx) == 0:
            miss += 1
        else:
            score += (int(hit_idx[0]) - r1) / (r2 - r1 + 1)
    return (score + miss) / nsim if nsim > 0 else np.nan

# ========== ES with min-epoch floor ==========
class EarlyStoppingWithMin(tf.keras.callbacks.EarlyStopping):
    def __init__(self, min_epochs=MIN_EPOCHS, **kwargs):
        super().__init__(**kwargs)
        self.min_epochs = min_epochs
    def on_epoch_end(self, epoch, logs=None):
        if (epoch + 1) < self.min_epochs:
            return
        return super().on_epoch_end(epoch, logs)

# ========== MAIN ==========
rng = np.random.RandomState(RNG_STATE)
summary_rows = []

# Load IF-driven splits once (supports explicit 60/20/20 if present)
print(f"Loading predefined IF splits:\n  VAL:  {VAL_SPLIT_CSV}\n  TEST: {TEST_SPLIT_CSV}")
by_signal = load_if_splits(VAL_SPLIT_CSV, TEST_SPLIT_CSV)

for S in SIGNALS:
    print(f"\n=== Signal {S} (LSTM AE; IF-driven splits; tail-only metrics) ===")
    Xdf, Ydf = load_signal(S)

    # Build sim list from all columns
    sims = []
    for sim_idx, col in enumerate(Xdf.columns):
        x = Xdf[col].to_numpy(dtype=float)
        y = Ydf[col].to_numpy(dtype=int)
        sim_name = f"sig{S}_sim{sim_idx}"
        sims.append(dict(x=x, y=y, name=sim_name, col_idx=sim_idx))

    # Pick train/val/test sims from the IF files (prefer explicit 'train' if provided)
    sig_splits = by_signal.get(S, {"train": set(), "val": set(), "test": set()})
    want_train = set(sig_splits.get("train", set()))
    want_val   = set(sig_splits.get("val", set()))
    want_test  = set(sig_splits.get("test", set()))

    train_sims, val_sims, test_sims = [], [], []
    missing_train, missing_val, missing_test = set(), set(), set()
    sim_by_name = {d["name"]: d for d in sims}

    use_explicit_train = len(want_train) > 0

    if use_explicit_train:
        for name in sorted(want_train):
            d = sim_by_name.get(name)
            if d is None: missing_train.add(name); continue
            if len(d["x"]) < (TRAIN_DAYS + TAIL_DAYS): continue
            train_sims.append(d)

    for name in sorted(want_val):
        d = sim_by_name.get(name)
        if d is None: missing_val.add(name); continue
        if len(d["x"]) < (TRAIN_DAYS + TAIL_DAYS): print(f"  ⚠ Skipping VAL {name}: too short"); continue
        val_sims.append(d)

    for name in sorted(want_test):
        d = sim_by_name.get(name)
        if d is None: missing_test.add(name); continue
        if len(d["x"]) < (TRAIN_DAYS + TAIL_DAYS): print(f"  ⚠ Skipping TEST {name}: too short"); continue
        test_sims.append(d)

    if not use_explicit_train:
        picked = {d["name"] for d in val_sims} | {d["name"] for d in test_sims}
        for d in sims:
            if d["name"] in picked: continue
            if len(d["x"]) >= (TRAIN_DAYS + TAIL_DAYS): train_sims.append(d)

    print(f"  Using splits (from IF files): {len(train_sims)} train, {len(val_sims)} val, {len(test_sims)} test")
    if missing_train: print(f"  ⚠ Missing in data (TRAIN): {sorted(missing_train)[:5]}{' ...' if len(missing_train)>5 else ''}")
    if missing_val:  print(f"  ⚠ Missing in data (VAL):   {sorted(missing_val)[:5]}{' ...' if len(missing_val)>5 else ''}")
    if missing_test: print(f"  ⚠ Missing in data (TEST):  {sorted(missing_test)[:5]}{' ...' if len(missing_test)>5 else ''}")

    # Optional confirmation dump
    pd.Series([d["col_idx"] for d in val_sims]).to_csv(f"lstm_val_indices_sig{S}.csv", index=False, header=False)
    pd.Series([d["col_idx"] for d in test_sims]).to_csv(f"lstm_test_indices_sig{S}.csv", index=False, header=False)

    if not train_sims:
        print("  No training sims with enough length; skipping."); continue
    if not test_sims:
        print("  No test sims with enough length; skipping."); continue

    # ---- Unsupervised TRAIN windows from first 6y
    Xtr_list = []
    for d in train_sims:
        x_train = d["x"][:TRAIN_DAYS]
        X_seq, _ = make_seq_labels(x_train, np.zeros_like(x_train), SEQ, STRIDE)
        if len(X_seq): Xtr_list.append(X_seq)
    if not Xtr_list:
        print("  No training windows; skipping."); continue
    Xtr = np.concatenate(Xtr_list).reshape(-1, SEQ, 1).astype(np.float32)
    print(f"  Train windows: {len(Xtr)} from first {TRAIN_YEARS} years")

    # ---- VALIDATION: build with (SEQ-1) days of CONTEXT so we get 1 score per tail day (no padding)
    Xv_list, Yv_list = [], []
    for d in val_sims:
        x = d["x"]; y = d["y"]
        tail_start = len(x) - TAIL_DAYS
        ctx_start  = tail_start - (SEQ - 1)
        if ctx_start < 0: continue
        x_ctx_tail = x[ctx_start : tail_start + TAIL_DAYS]
        y_ctx_tail = y[ctx_start : tail_start + TAIL_DAYS]
        Xv, Yv = make_seq_labels(x_ctx_tail, y_ctx_tail, SEQ, STRIDE)  # len == TAIL_DAYS
        if len(Xv) == TAIL_DAYS:
            Xv_list.append(Xv); Yv_list.append(Yv)
    Xval = np.concatenate(Xv_list).reshape(-1, SEQ, 1).astype(np.float32) if Xv_list else np.empty((0,SEQ,1), np.float32)
    Yval = np.concatenate(Yv_list).astype(int) if Yv_list else np.empty((0,), np.int32)
    if len(Yval):
        pos = int(Yval.sum()); pct = 100*Yval.mean()
        print(f"  Val windows (tail, full coverage): {len(Xval)}  positives: {pos} ({pct:.1f}%)")
    else:
        print("  No val windows (tuner will fall back to train guard-rail).")

    # ---- TEST: same context trick → exactly TAIL_DAYS scores per sim, no front padding
    per_sim_labels = []   # each length = TAIL_DAYS
    A_tail_cols    = []   # alarms per sim (343)
    O_tail_cols    = []   # labels per sim (343)
    Xte_chunks     = []   # each chunk length = TAIL_DAYS (of SEQ-length windows)

    for d in test_sims:
        x = d["x"]; y = d["y"]
        tail_start = len(x) - TAIL_DAYS
        ctx_start  = tail_start - (SEQ - 1)
        if ctx_start < 0: continue
        x_ctx_tail = x[ctx_start : tail_start + TAIL_DAYS]
        y_tail     = y[tail_start : tail_start + TAIL_DAYS].astype(int)  # 343 labels aligned to tail days
        Xte, _     = make_seq_labels(x_ctx_tail, y[ctx_start : tail_start + TAIL_DAYS], SEQ, STRIDE)  # len == 343
        if len(Xte) == TAIL_DAYS:
            Xte_chunks.append(Xte)
            per_sim_labels.append(y_tail)

    if not Xte_chunks:
        print("  No test windows; skipping."); continue

    Xte = np.concatenate(Xte_chunks).reshape(-1, SEQ, 1).astype(np.float32)
    print(f"  Test windows (tail, full coverage): {len(Xte)}")

    # ---- Build & train
    model = build_lstm_autoencoder(SEQ, LEARNING_RATE)
    if len(Xval):
        cbs = [
            EarlyStoppingWithMin(min_epochs=MIN_EPOCHS, patience=PATIENCE, restore_best_weights=True, monitor="val_loss"),
            tf.keras.callbacks.ReduceLROnPlateau(monitor="val_loss", factor=0.5, patience=5, min_lr=1e-6),
            tf.keras.callbacks.TerminateOnNaN(),
        ]
        model.fit(Xtr, Xtr, validation_data=(Xval, Xval), epochs=EPOCHS, batch_size=BATCH_SIZE, callbacks=cbs, verbose=1)
    else:
        cbs = [
            EarlyStoppingWithMin(min_epochs=MIN_EPOCHS, patience=PATIENCE, restore_best_weights=True, monitor="loss"),
            tf.keras.callbacks.ReduceLROnPlateau(monitor="loss", factor=0.5, patience=5, min_lr=1e-6),
            tf.keras.callbacks.TerminateOnNaN(),
        ]
        model.fit(Xtr, Xtr, epochs=EPOCHS, batch_size=BATCH_SIZE, callbacks=cbs, verbose=1)

    # ---- Validation: tune mix + threshold (or fallback if no val)
    if len(Xval):
        val_rec = model.predict(Xval, batch_size=BATCH_SIZE, verbose=0)
        best = tune_mix_and_threshold(
            Xval, Yval, val_rec,
            mixes=MIX_GRID, spec_target=SPEC_TARGET,
            w_sens=TUNE_WEIGHT_SENS, w_spec=TUNE_WEIGHT_SPEC
        )
        AM_MSE, AM_MAX = best["mix"]
        best_threshold = best["threshold"]

        # Guard-rail on TRAIN with chosen mix
        tr_rec_for_guard = model.predict(Xtr, batch_size=BATCH_SIZE, verbose=0)
        tr_mse_g = np.mean((Xtr - tr_rec_for_guard)**2, axis=(1,2))
        tr_max_g = np.max(np.abs(Xtr - tr_rec_for_guard), axis=(1,2))
        tr_scores_g = AM_MSE*tr_mse_g + AM_MAX*tr_max_g
        guardrail_thr = float(np.percentile(tr_scores_g, 95))
        if best_threshold < guardrail_thr:
            print(f"  Guard-rail raised threshold: {best_threshold:.6f} -> {guardrail_thr:.6f}")
            best_threshold = guardrail_thr

        print(f"  Chosen mix: MSE={AM_MSE:.2f}, MAX={AM_MAX:.2f} | "
              f"threshold={best_threshold:.6f} | val Sens={best['sens']:.3f}, Spec={best['spec']:.3f}")
    else:
        tr_rec = model.predict(Xtr, batch_size=BATCH_SIZE, verbose=0)
        tr_mse = np.mean((Xtr - tr_rec)**2, axis=(1,2))
        best_threshold = float(np.percentile(tr_mse, 95))
        AM_MSE, AM_MAX = ANOMALY_MIX_MSE, ANOMALY_MIX_MAX
        print(f"  No validation; using default mix MSE={AM_MSE:.2f}, MAX={AM_MAX:.2f} and 95th pct train MSE threshold.")

    # ---- TEST inference (one score per tail day per sim; no padding)
    te_rec  = model.predict(Xte, batch_size=BATCH_SIZE, verbose=0)
    te_mse  = np.mean((Xte - te_rec)**2, axis=(1,2))
    te_max  = np.max(np.abs(Xte - te_rec), axis=(1,2))
    te_scores = AM_MSE*te_mse + AM_MAX*te_max  # length = TAIL_DAYS * n_test_sims

    # Chunk back per sim (each chunk length = TAIL_DAYS)
    ofs = 0
    for y_tail in per_sim_labels:
        yhat_seq = (te_scores[ofs:ofs+TAIL_DAYS] >= best_threshold).astype(int)
        ofs += TAIL_DAYS
        A_tail_cols.append(yhat_seq)           # length 343
        O_tail_cols.append(y_tail.astype(int)) # length 343

    if not A_tail_cols:
        print("  No test sims; skipping metrics."); continue

    A_tail = np.column_stack(A_tail_cols)  # [343, nsim_test]
    O_tail = np.column_stack(O_tail_cols)  # [343, nsim_test]

    # ---- Tail-only metrics
    sens = compute_sensitivity_tail(A_tail, O_tail)
    spec = compute_specificity_tail(A_tail, O_tail)
    fpr  = compute_fpr_tail(A_tail, O_tail)
    pod  = compute_pod_tail(A_tail, O_tail)
    tim  = compute_timeliness_tail(A_tail, O_tail)

    print(f"  TAIL-ONLY → Sens={sens:.3f}, Spec={spec:.3f}, FPR={fpr:.3f}, POD={pod:.3f}, Tim={tim:.3f}")

    summary_rows.append(dict(
        signal=S, sensitivity=sens, specificity=spec, fpr=fpr, pod=pod, timeliness=tim,
        threshold=best_threshold, mix_mse=AM_MSE, mix_max=AM_MAX, nsim_test=A_tail.shape[1]
    ))

# ========== SAVE SUMMARY ==========
if summary_rows:
    df = pd.DataFrame(summary_rows).set_index("signal").sort_index()
    print("\n=== LSTM Autoencoder — Tail-only Comparator Metrics (by signal) ===")
    print(df.round(6))
    print("\n=== Means ===")
    print(df[["sensitivity","specificity","fpr","pod","timeliness"]].mean().round(6))
    df.to_csv("LSTM_AE_tail_only_metrics.csv")
    print("\nSaved: LSTM_AE_tail_only_metrics.csv")
else:
    print("\nNo results to summarize.")


GPU Available: [PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]

=== Signal 1 (LSTM AE; tail-only metrics) ===
  Saved indices: val=200 → lstm_val_indices_sig1.csv,  test=200 → lstm_test_indices_sig1.csv
  Train windows: 1302600 from first 6 years
  Val windows (tail): 66000  positives: 3366 (5.1%)
  Test windows (tail): 66000
Epoch 1/60
10177/10177 ━━━━━━━━━━━━━━━━━━━━ 334s 31ms/step - loss: 156.6994 - mse: 97330.2344 - val_loss: 74.5295 - val_mse: 12205.1758 - learning_rate: 0.0010
Epoch 2/60
10177/10177 ━━━━━━━━━━━━━━━━━━━━ 319s 31ms/step - loss: 42.2211 - mse: 5972.9272 - val_loss: 59.1011 - val_mse: 7628.2207 - learning_rate: 0.0010
Epoch 3/60
10177/10177 ━━━━━━━━━━━━━━━━━━━━ 319s 31ms/step - loss: 39.4475 - mse: 4800.7227 - val_loss: 85.9164 - val_mse: 30516.6133 - learning_rate: 0.0010
Epoch 4/60
10177/10177 ━━━━━━━━━━━━━━━━━━━━ 320s 31ms/step - loss: 38.4455 - mse: 4481.5835 - val_loss: 33.8840 - val_mse: 3483.3225 - learning_rate: 0.0010
Epoch 5/60
10177/1017